In [ ]:
from collections import defaultdict
from surprise import Dataset
import pandas as pd
from surprise import SVD
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import numpy as np
from surprise import dump
import os

In [ ]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def train_and_save_model () :
    #train and save model

    df = pd.read_csv('testForInput.csv')
    # A reader is still needed but only the rating_scale param is requiered.
    reader = Reader(rating_scale=(1, 5))

    # The columns must correspond to user id, item id and ratings (in that order).
    data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

    #train data with algorithm
    algo = SVD()
    cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

    # Dump algorithm and reload it.
    file_name = os.path.expanduser('SVD_dump_file')
    dump.dump(file_name, algo=algo)
    
def load_model_from_disk():
    df = pd.read_csv('testForInput.csv')
    file_name = os.path.expanduser('SVD_dump_file')
    _, loaded_algo = dump.load(file_name)
    return loaded_algo

In [ ]:
loaded_algo = load_model_from_disk()

In [ ]:
from flask import Flask
app = Flask(__name__)
#'ALOXOO497B4LH'
@app.route('/<user_name>')
def getTopNitems(user_name):
    
    test = df[df['userID'] == user_name].drop(df.columns[3], axis=1)
    columnsTitles=["userID","itemID","rating"]
    test=test.reindex(columns=columnsTitles)
    predictions = loaded_algo.test(np.array(test))

    top_n = get_top_n(predictions, n=10)
    ret = []
    # Print the recommended items for each user
    for uid, user_ratings in top_n.items():
        ret.append ( (uid, [iid for (iid, _) in user_ratings]) )
    return str(ret[0])    
        

if __name__ == '__main__':
   app.run()